# DatabricksVectorSearch

[Databricks Vector Search](https://docs.databricks.com/en/generative-ai/vector-search.html) 是一个无服务器的相似性搜索引擎，允许您在向量数据库中存储数据的向量表示，包括元数据。借助 Vector Search，您可以从 Unity Catalog 管理的 Delta 表创建自动更新的向量搜索索引，并通过简单的 API 查询它们以返回最相似的向量。

本笔记本演示了如何将 LangChain 与 Databricks Vector Search 结合使用。

## 设置

要访问 Databricks 模型，您需要创建一个 Databricks 账户，设置凭据（仅当您不在 Databricks 工作空间内时），并安装所需的包。

### 凭据（仅当您不在 Databricks 工作空间内时）

如果您在 Databricks 内运行 LangChain 应用，则可以跳过此步骤。

否则，您需要分别将 Databricks 工作空间主机名和个人访问令牌手动设置为 `DATABRICKS_HOST` 和 `DATABRICKS_TOKEN` 环境变量。有关如何获取访问令牌的说明，请参阅[认证文档](https://docs.databricks.com/en/dev-tools/auth/index.html#databricks-personal-access-tokens)。

In [1]:
import getpass
import os

os.environ["DATABRICKS_HOST"] = "https://your-databricks-workspace"
if "DATABRICKS_TOKEN" not in os.environ:
    os.environ["DATABRICKS_TOKEN"] = getpass.getpass(
        "Enter your Databricks access token: "
    )

### 安装

LangChain Databricks 集成位于 `databricks-langchain` 包中。

In [ ]:
%pip install -qU databricks-langchain

### 创建向量搜索端点和索引（如果尚未创建）

在本节中，我们将使用客户端 SDK 创建一个 Databricks 向量搜索端点和一个索引。

如果您已经拥有端点和索引，可以跳过本节，直接进入“实例化”部分。

首先，实例化 Databricks VectorSearch 客户端：

In [ ]:
from databricks.vector_search.client import VectorSearchClient

client = VectorSearchClient()

接下来，我们将创建一个新的 VectorSearch endpoint。

In [14]:
endpoint_name = "<your-endpoint-name>"

client.create_endpoint(name=endpoint_name, endpoint_type="STANDARD")

最后，我们将创建一个可以在端点上查询的索引。Databricks Vector Search 中有两种类型的索引，`DatabricksVectorSearch` 类支持这两种用例。

*   **Delta Sync Index** 会自动与源 Delta 表同步，并在 Delta 表中的底层数据发生变化时自动增量更新索引。

*   **Direct Vector Access Index** 支持直接读写向量和元数据。用户负责使用 REST API 或 Python SDK 更新此表。

此外，对于 delta-sync 索引，您可以选择使用 Databricks 管理的嵌入（embeddings）或自管理的嵌入（通过 LangChain embeddings 类）。

以下代码创建了一个**直接访问**索引。有关创建其他类型索引的说明，请参阅 [Databricks 文档](https://docs.databricks.com/en/generative-ai/create-query-vector-search.html)。

In [ ]:
index_name = "<your-index-name>"  # Format: "<catalog>.<schema>.<index-name>"

index = client.create_direct_access_index(
    endpoint_name=endpoint_name,
    index_name=index_name,
    primary_key="id",
    # Dimension of the embeddings. Please change according to the embedding model you are using.
    embedding_dimension=3072,
    # A column to store the embedding vectors for the text data
    embedding_vector_column="text_vector",
    schema={
        "id": "string",
        "text": "string",
        "text_vector": "array<float>",
        # Optional metadata columns
        "source": "string",
    },
)

index.describe()

## 实例化

`DatabricksVectorSearch` 的实例化方式会略有不同，具体取决于您的索引使用的是 Databricks 管理的嵌入还是自带的嵌入，即您选择的 LangChain Embeddings 对象。

如果你在使用 Databricks 管理的 embedding 的增量同步索引：

In [ ]:
from databricks_langchain import DatabricksVectorSearch

vector_store = DatabricksVectorSearch(
    endpoint=endpoint_name,
    index_name=index_name,
)

如果您正在使用直接访问索引或 Delta-Sync 索引，并结合自管理的嵌入，那么您还需要在源表中提供用于嵌入的 embedding model 和 text column：

import EmbeddingTabs from "@theme/EmbeddingTabs";

<EmbeddingTabs/>

In [ ]:
# | output: false
# | echo: false
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

In [ ]:
vector_store = DatabricksVectorSearch(
    endpoint=endpoint_name,
    index_name=index_name,
    embedding=embeddings,
    # The column name in the index that contains the text data to be embedded
    text_column="document_content",
)

## 管理向量存储

### 向向量存储添加项目

注意：通过 `add_documents` 方法向向量存储添加项目仅支持**直接访问**索引。

In [20]:
from langchain_core.documents import Document

document_1 = Document(page_content="foo", metadata={"source": "https://example.com"})

document_2 = Document(page_content="bar", metadata={"source": "https://example.com"})

document_3 = Document(page_content="baz", metadata={"source": "https://example.com"})

documents = [document_1, document_2, document_3]

vector_store.add_documents(documents=documents, ids=["1", "2", "3"])

['1', '2', '3']

### 从向量存储中删除条目

注意：通过 `delete` 方法删除到向量存储的条目仅支持 **直接访问（**direct-access**）** 索引。

In [21]:
vector_store.delete(ids=["3"])

True

## 查询向量存储

一旦你的向量存储创建完成并且已添加了相关文档，在运行你的链或代理时，你最有可能想查询它。

### 直接查询

可以通过以下方式执行简单的相似性搜索：

In [23]:
results = vector_store.similarity_search(
    query="thud", k=1, filter={"source": "https://example.com"}
)
for doc in results:
    print(f"* {doc.page_content} [{doc.metadata}]")

* foo [{'id': '1'}]


注意：默认情况下，相似性搜索仅返回主键和文本列。如果您想检索与文档关联的自定义元数据，请在初始化向量存储时在 `columns` 参数中传入额外的列。

In [35]:
vector_store = DatabricksVectorSearch(
    endpoint=endpoint_name,
    index_name=index_name,
    embedding=embeddings,
    text_column="text",
    columns=["source"],
)

results = vector_store.similarity_search(query="thud", k=1)
for doc in results:
    print(f"* {doc.page_content} [{doc.metadata}]")

* foo [{'source': 'https://example.com', 'id': '1'}]


如果你想执行相似性搜索并接收相应的得分，可以运行：

In [36]:
results = vector_store.similarity_search_with_score(
    query="thud", k=1, filter={"source": "https://example.com"}
)
for doc, score in results:
    print(f"* [SIM={score:3f}] {doc.page_content} [{doc.metadata}]")

* [SIM=0.414035] foo [{'source': 'https://example.com', 'id': '1'}]


### 通过转化为检索器进行查询

你也可以将向量存储转化为检索器以便在你的链中更方便地使用。

In [37]:
retriever = vector_store.as_retriever(search_type="mmr", search_kwargs={"k": 1})
retriever.invoke("thud")

[Document(metadata={'source': 'https://example.com', 'id': '1'}, page_content='foo')]

## 使用检索增强生成（Retrieval-Augmented Generation）

有关如何将此向量存储用于检索增强生成（RAG）的指南，请参阅以下部分：

- [教程](/docs/tutorials/)
- [操作指南：通过 RAG 进行问答](https://python.langchain.com/docs/how_to/#qa-with-rag)
- [检索概念文档](https://python.langchain.com/docs/concepts/retrieval)

## API 参考

如需了解 DatabricksVectorSearch 所有功能和配置的详细文档，请访问 API 参考：https://api-docs.databricks.com/python/databricks-ai-bridge/latest/databricks_langchain.html#databricks_langchain.DatabricksVectorSearch